In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

In [2]:
followers = pd.read_csv("giant_csv_of_followers.csv",dtype = {"user":str,"id":str})

In [3]:
followers.drop("id",axis = 1,inplace = True)
followers.set_index("user",inplace = True)
followers.head()

,9NEWS,Boulder_Bcycle,BuiltInColorado,CBSDenver,CuredBoulder,dailycamera,DenverChannel,EVOLfoods,Elaine,HotelBoulderado,...,oakatfourteenth,oskarblues,thekitchen,upslope,ericmbudd_follows,created_at,followers,following,screen_name,tweets_count
user,,,,,,,,,,,,,,,,,,,,,
543200580,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2012-04-02 04:49:06,16.0,43.0,ntmadrid,79.0
2418502076,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014-03-30 05:15:49,36.0,104.0,heidi_heidisk,257.0
2378927935,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014-03-08 15:52:32,4.0,46.0,manda_spahr,3.0
816638395011178496,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2017-01-04 13:32:20,21.0,126.0,fastablasta,17.0
2479473494,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014-05-06 08:12:59,857.0,1795.0,JJCRockin,1901.0


In [4]:
followers["user_degree"] = followers[followers.columns[0:99]].apply(sum,axis = 1)

In [5]:
followers_higher_degree = followers[followers["user_degree"] > 1]

In [ ]:
edge_list = []
for column in followers_higher_degree.columns[0:99]:
    edge_list.extend([(x,column) for x in followers_higher_degree[followers_higher_degree[column] == 1].index.tolist()])

In [7]:
influencers = {x:{"is_influencer":True} for x in followers_higher_degree.columns[0:99].tolist()}

In [ ]:
followers_higher_degree["is_influencer"] = False

In [ ]:
users = followers_higher_degree[["ericmbudd_follows","followers","following","tweets_count","is_influencer"]].transpose().to_dict()

In [ ]:
digraph = nx.DiGraph()
digraph.add_nodes_from(influencers)
digraph.add_nodes_from(users)

In [ ]:
digraph.add_edges_from(edge_list)

In [ ]:
eric_follows = followers_higher_degree[followers_higher_degree["ericmbudd_follows"] == 1].index.tolist()
g_eric_follows = digraph.subgraph(eric_follows + list(influencers.keys()))

In [26]:
influencers_dict = {k:v for k,v in zip(range(0,100), followers_higher_degree[1:100].columns.tolist())}

In [28]:
followers_higher_degree.loc[:,"edge_list"] = followers_higher_degree[list(influencers.keys())].apply(
    lambda x: ",".join([influencers_dict[i] for i,j in enumerate(x.tolist()) if j == 1]),axis = 1)

/mnt/home/fiona/.virtualenvs/graph-analysis/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [40]:
test = (followers_higher_degree[["edge_list","ericmbudd_follows"]]
        .reset_index()
        .groupby("edge_list")
        .agg({"user": pd.Series.nunique, "ericmbudd_follows": sum})
        .sort_values(by = "user",ascending = False))

In [43]:
test.sort_values(by = "ericmbudd_follows", ascending = False)

,ericmbudd_follows,user
edge_list,,
"CUBoulderAlumni,thekitchen",17.0,31
"CUBoulderAlumni,jvaleski",12.0,17
"BoulderWalkBike,CUBoulderAlumni",10.0,16
"BoulderWalkBike,thekitchen",7.0,20075
"CUBoulderAlumni,DenverWestword",7.0,8
"CUBoulderAlumni,denbizjournal",6.0,6
"CUBoulderAlumni,gracekboyle",6.0,8
"CUBoulderAlumni,jaredpolis",6.0,7
"CUBoulderAlumni,downtownboulder",6.0,9
